# ont_rdb_explorer

## Import packages
Also set the path to ont_rdb and the name of your snakemake profile for processing purposes.

In [ ]:
#import informant_class
#import swifter
from informant_class import *
# Adjust this to the directory storing the ont_rdb package.
ont_rdb_path = "/home/groups/CEDAR/franksto/ont_rdb/ont_rdb"
# Set the snakemake profile you wish to use:
snakemake_profile = 'mamba'

from IPython.display import display, clear_output
import ipywidgets as widgets
import os
import importlib
import sys
print('Completed')

## Select and import ontology module from drop-down menu.


In [ ]:
selected_module_name = None
imported_module = None

def list_files_in_folder(folder_path):
    """Lists files in the given folder path."""
    files = [f for f in os.listdir(folder_path) if (os.path.isfile(os.path.join(folder_path, f)) and f.endswith("_ontology.py"))]
    return files

ontologies_folder = ont_rdb_path + '/ontologies'
file_list = list_files_in_folder(ontologies_folder)
dropdown_menu = widgets.Dropdown(options=file_list, description='Files:', disabled=False)
display(dropdown_menu)

button = widgets.Button(description="Import Selected Ontology Module", layout=widgets.Layout(width='auto'))
display(button)

output = widgets.Output()

@button.on_click
def button_on_click(b):
    global selected_module_name
    
    global imported_module
    with output:
        clear_output()
        selected_file = dropdown_menu.value
        module_name = selected_file[:-3]  # Remove the '.py' extension
        import_path = f'ontologies.{module_name}'
        
        try:
            # Dynamically import the selected module
            imported_module = importlib.import_module(import_path)
            # Optionally, add the imported module to sys.modules
            sys.modules[module_name] = imported_module
            print(f"Successfully imported {module_name}.")
            selected_module_name = module_name
        except Exception as e:
            print(f"Failed to import {module_name}: {e}.")

display(output)



In [30]:
imported_module

<module 'ontologies.project_manager_version_1_ontology' from '/home/groups/CEDAR/franksto/ont_rdb/ont_rdb/ontologies/project_manager_version_1_ontology.py'>

### Import informant classes from ontology module and construct or access its associated digraph dataframe.
If necessary, modify the ``command`` to use appropriate snakemake configurations.

For example:
* ``use-conda: true``
* ``conda-frontend: mamba``
* ``cores: 1``

In [12]:
import subprocess

# Run Snakemake unlock command
unlock_command = f"snakemake --profile {snakemake_profile} --unlock"
print(f"🔓 Unlocking Snakemake: {unlock_command}")
unlock_result = subprocess.run(unlock_command, shell=True, capture_output=True, text=True)

# Check if unlock was successful
if unlock_result.returncode == 0:
    print("✅ Snakemake unlocked successfully.")
else:
    print(f"❌ Unlock failed. Error:\n{unlock_result.stderr}")
    exit(1)

🔓 Unlocking Snakemake: snakemake --profile mamba --unlock
✅ Snakemake unlocked successfully.


In [4]:
import pandas as pd
import subprocess
import snakemake
command = f"snakemake --profile {snakemake_profile} ontology_dataframes/{selected_module_name}_dataframe.pkl"

print(f"Executing: {command}")
result = subprocess.run(command, shell=True)
#result = subprocess.run(command, capture_output=True, text=True, shell=True)

# Check if Snakemake succeeded
if result.returncode == 0:
    print(f"\n✅ {selected_module_name} dataframe successfully constructed.")
else:
    print(f"\n❌ Snakemake failed. Check {log_file} for details.\nError:\n", result.stderr)
    exit(1)


Executing: snakemake --profile mamba ontology_dataframes/2024-10-14_hic_vae_ontology_dataframe.pkl


Using profile mamba for setting default command line arguments.
Assuming unrestricted shared filesystem usage.
Building DAG of jobs...

KeyboardInterrupt



In [13]:
ontology_dataframe = pd.read_pickle(ont_rdb_path + '/ontology_dataframes/' + selected_module_name + '_dataframe.pkl')

selected_module = importlib.import_module(selected_module_name)
for name in dir(selected_module):
    if not name.startswith('_'):  # Skip internal names
        globals()[name] = getattr(selected_module, name)

## Explore your ontology and create your database.
Construct and save informants and informant dataframes to organize  objects in the context of your ontology. 

In [14]:
ontology_dataframe

,informant_subclass_name,informant_subclass,direct_parent_indices,direct_child_indices,is_sink,source_depth,sink_depth,to_nearest_sink
0,Informant,<class 'informant_class.Informant'>,[],"[1, 3, 4, 5, 6, 8, 9, 10, 11, 17]",0,0,1,"[3, 4, 5, 6, 9, 11]"
1,Directory_Informant,<class 'informant_class.Directory_Informant'>,[0],"[2, 7, 18, 19]",0,1,1,"[7, 18]"
2,File_Informant,<class 'informant_class.File_Informant'>,[1],"[15, 16, 20, 25]",0,2,1,"[15, 16, 25]"
3,Informant_Dataframe,<class 'informant_class.Informant_Dataframe'>,[0],[],1,1,0,[]
4,DataBase,<class 'ontologies.project_manager_version_1_o...,[0],[],1,1,0,[]
5,Algorithm,<class 'ontologies.project_manager_version_1_o...,[0],[],1,1,0,[]
6,Parameters,<class 'ontologies.project_manager_version_1_o...,[0],[],1,1,0,[]
7,File_Set,<class 'ontologies.project_manager_version_1_o...,[1],[],1,2,0,[]
8,Institution,<class 'ontologies.project_manager_version_1_o...,[0],"[12, 13]",0,1,1,"[12, 13]"
9,Article,<class 'ontologies.project_manager_version_1_o...,[0],[],1,1,0,[]


In [15]:
#!pip install networkx
#!pip install pyvis


In [17]:
import networkx as nx
from pyvis.network import Network
import pandas as pd
import os

# Load ontology dataframe
ontology_df = ontology_dataframe  # Assuming it's already loaded

# Create a directed graph
G = nx.DiGraph()

# Add nodes with labels
for _, row in ontology_df.iterrows():
    node = row['informant_subclass_name']
    G.add_node(node, title=node)  # title is used for hover tooltip

# Add edges based on parent-child relationships
for _, row in ontology_df.iterrows():
    parent_indices = row['direct_parent_indices']
    child = row['informant_subclass_name']
    for parent_index in parent_indices:
        parent = ontology_df.iloc[parent_index]['informant_subclass_name']
        G.add_edge(parent, child)

# Create an interactive visualization using pyvis
net = Network(height="750px", width="100%", directed=True, notebook=True)

# Convert NetworkX graph to Pyvis
net.from_nx(G)

# Customize node appearance
for node in net.nodes:
    node['size'] = G.out_degree(node['id']) * 3  # Scale by number of children
    node['color'] = 'red' if G.out_degree(node['id']) > 5 else 'blue'  # Highlight hubs
    node['title'] = f"{node['id']} (Children: {G.out_degree(node['id'])})"

# Save and open the interactive visualization
output_path = f"{selected_module_name}_graph.html"
net.save_graph(output_path)
print(f"✅ Interactive ontology graph saved to {output_path}")

# Open in browser automatically (only works if running locally)
if os.path.exists(output_path):
    import webbrowser
    webbrowser.open(f"file://{os.path.abspath(output_path)}")


✅ Interactive ontology graph saved to project_manager_version_1_ontology_graph.html


In [3]:
import networkx as nx
from pyvis.network import Network
import re
import os
import sys

def extract_functions(script_content):
    """Extracts function names from a Python script."""
    function_pattern = re.compile(r'def (\w+)\(')
    return function_pattern.findall(script_content)

def extract_function_calls(script_content, function_names):
    """Extracts function calls from a Python script, limited to known functions."""
    call_pattern = re.compile(r'(\w+)\(')
    calls = call_pattern.findall(script_content)
    return [call for call in calls if call in function_names]

def build_dependency_graph(script_path):
    """Builds a function dependency graph from a Python script."""
    if not os.path.exists(script_path):
        print(f"❌ Error: The script '{script_path}' does not exist.")
        return None

    with open(script_path, 'r', encoding='utf-8') as f:
        script_content = f.read()

    function_names = extract_functions(script_content)

    # Build the graph
    G = nx.DiGraph()

    for func in function_names:
        G.add_node(func)

    function_blocks = re.split(r'def (\w+)\(', script_content)[1:]  # Split by function definitions
    for i in range(0, len(function_blocks), 2):
        func_name = function_blocks[i]
        func_body = function_blocks[i + 1] if i + 1 < len(function_blocks) else ""
        called_functions = extract_function_calls(func_body, function_names)
        for called_func in called_functions:
            G.add_edge(func_name, called_func)

    return G

def generate_interactive_graph(G, output_path="dependency_graph.html"):
    """Generates an interactive dependency graph using pyvis."""
    if G is None or len(G.nodes) == 0:
        print("⚠️ No functions detected. Ensure your script has function definitions.")
        return

    net = Network(height="750px", width="100%", directed=True, notebook=False)
    net.from_nx(G)

    # Customize node appearance
    for node in net.nodes:
        node['size'] = G.out_degree(node['id']) * 5  # Scale by number of calls
        node['color'] = 'red' if G.out_degree(node['id']) > 5 else 'blue'
        node['title'] = f"{node['id']} (Calls: {G.out_degree(node['id'])})"

    # Save and open the interactive visualization
    net.save_graph(output_path)
    print(f"✅ Interactive dependency graph saved to {output_path}")

    # Open in browser (only if running locally)
    if os.path.exists(output_path):
        import webbrowser
        webbrowser.open(f"file://{os.path.abspath(output_path)}")

if __name__ == "__main__":
    if len(sys.argv) < 2:
        print("Usage: python dependency_graph.py <path_to_python_script>")
        sys.exit(1)

    script_path = '/home/groups/CEDAR/franksto/ont_rdb/ont_rdb/informant_class.py'#sys.argv[1]
    G = build_dependency_graph(script_path)
    generate_interactive_graph(G)


✅ Interactive dependency graph saved to dependency_graph.html


## Compute Most Specific Generalizations (Least Common Ancestors) of Any List of Informant Classes

In [16]:
from collections import deque, defaultdict

df= ontology_dataframe
# Build parent map and depth map
parent_map = defaultdict(list)
depth_map = {}

for idx, row in df.iterrows():
    depth_map[row['informant_subclass_name']] = row['source_depth']
    for parent in row['direct_parent_indices']:
        parent_map[row['informant_subclass_name']].append(df.iloc[parent]['informant_subclass_name'])

# Helper function to find all ancestors
# Helper function to find all ancestors
def find_ancestors(node, parent_map):
    queue = deque([node])
    ancestors = set()
    while queue:
        current = queue.popleft()
        ancestors.add(current)
        for parent in parent_map[current]:
            if parent not in ancestors:
                queue.append(parent)
    return ancestors

# Helper function to compute unique LCA for two nodes
def find_lca_two_nodes(node1, node2, parent_map, depth_map):
    queue1 = deque([node1])
    queue2 = deque([node2])
    
    visited1 = set()
    visited2 = set()
    
    while queue1 or queue2:
        if queue1:
            current1 = queue1.popleft()
            if current1 in visited2:
                return current1
            visited1.add(current1)
            for parent in parent_map[current1]:
                if parent not in visited1:
                    queue1.append(parent)
        
        if queue2:
            current2 = queue2.popleft()
            if current2 in visited1:
                return current2
            visited2.add(current2)
            for parent in parent_map[current2]:
                if parent not in visited2:
                    queue2.append(parent)
    
    return None

# Recursive function to compute unique LCA for a list of nodes
def find_lca_list(nodes, parent_map, depth_map):
    if len(nodes) == 1:
        return nodes[0]
    
    if len(nodes) == 2:
        return find_lca_two_nodes(nodes[0], nodes[1], parent_map, depth_map)
    
    mid = len(nodes) // 2
    left_lca = find_lca_list(nodes[:mid], parent_map, depth_map)
    right_lca = find_lca_list(nodes[mid:], parent_map, depth_map)
    
    return find_lca_two_nodes(left_lca, right_lca, parent_map, depth_map)

# Find all ancestors of each node at the same depth as the LCA
def filter_lcas(lca, nodes, parent_map, depth_map):
    lca_depth = depth_map[lca]
    all_ancestors = set()
    
    # Collect all ancestors at the same depth as LCA
    for node in nodes:
        ancestors = find_ancestors(node, parent_map)
        for ancestor in ancestors:
            if depth_map[ancestor] == lca_depth:
                all_ancestors.add(ancestor)
    
    # Filter ancestors to retain only those that are common ancestors
    valid_lcas = set()
    for ancestor in all_ancestors:
        if all(ancestor in find_ancestors(node, parent_map) for node in nodes):
            valid_lcas.add(ancestor)
    
    return valid_lcas


In [27]:
# Example usage
nodes = ['ChIP_seq_bigWigAverage_Over_Bed_File',"Bed_File", "bigWig_File", "HiC_File", "Genome_Assembly"]
unique_lca = find_lca_list(nodes, parent_map, depth_map)
all_lcas = filter_lcas(unique_lca, nodes, parent_map, depth_map)

#print(f'The unique LCA of {nodes} is: {unique_lca}')
print(f'The lowest common ancestors of {nodes} are: {all_lcas}')

The lowest common ancestors of ['ChIP_seq_bigWigAverage_Over_Bed_File', 'Bed_File', 'bigWig_File', 'HiC_File', 'Genome_Assembly'] are: {'Computational_Bio_Source'}


In [11]:
# Initialize an empty informant dataframe object
my_informant_dataframe = informant_class.Informant_Dataframe()

In [26]:
pd.DataFrame([HiC_TAD_Boundary_Caller().__dict__])

,name,description,tags,reference_informant_names,informant_class,reference_informant_name_redundancy_values,source_depth,parameter_descriptions,featured_object_type,feature_type
0,None,None,[],[],HiC_TAD_Boundary_Caller,{},4,None,3D-Genome,TAD_Boundary


In [15]:
TopDom = TAD_Caller()

Parameters(
    name='Bungus',
    algorithm='TopDom',
).__dict__

{'name': 'Bungus',
 'description': None,
 'tags': [],
 'reference_informant_names': [],
 'informant_class': 'Parameters',
 'reference_informant_name_redundancy_values': {},
 'source_depth': 1,
 'algorithm': 'TopDom',
 'parameter_descriptions': {}}

In [10]:

# Initialize a default BedPe_File informant
bedpe_inf = BedPe_File()
                           
# Use the object's dictionary to see default, characteristic attributes/fields for this class of informant as defined in the ontology.
print(bedpe_inf.__dict__)

# Populate the fields for this informant by updating its dictionary.
bedpe_inf.__dict__.update({'name':"ENCFF661SAZ.bedpe",
'description': 'Basic loops file from ENCODE.',
'species': 'homo_sapiens',
'location': "/home/cfrankston/Projects/Auxiliaries/bedpe_tools/bedpe_data/ENCFF661SAZ.bedpe",
'genome_assembly_name': "GRCh38",
'gz':False})

# Observe that the fields have been populated
bedpe_inf.__dict__

# Append this informant to the empty informant dataframe
my_informant_dataframe.append([bedpe_inf])

# Observe that the informant dataframe now contains the informant
my_informant_dataframe.df

# Test filtering capabilities of the informant dataframe.
my_informant_dataframe.filter("(@genome_assembly_name == 'GRCh38') & (@gz == False)")

{'name': None, 'description': None, 'tags': [], 'reference_informant_names': [], 'informant_class': 'BedPe_File', 'reference_informant_name_redundancy_values': {}, 'source_depth': 5, 'species': None, 'location': None, 'external_locations': None, 'file_type': '.bedpe', 'genome_assembly_name': None, 'aliases': None, 'gz': None}


,name,informant,entry_time,verification_status
0,ENCFF661SAZ.bedpe,<ontologies.hic_January_24_2024_ontology.BedPe...,06_28_2024,pending


In [21]:
my_informant_dataframe.df.iloc[0]['informant'].__dict__['location']

'/home/cfrankston/Projects/Auxiliaries/bedpe_tools/bedpe_data/ENCFF661SAZ.bedpe'

In [22]:
this_informant_df = informant_class.Informant_Dataframe()

informants_list = []
informants_list.append(HiC_File(name='EZH2_KO_Merge.hic',
                  description='Bulk HiC data from the laboratory of Ted Braun at OHSU of hematopoietic stem cells after an EZH2 CRISPR knockout, merged from three technical replicates by the HiCkory authored by PhD. student Benjamin Skubi in the Yardimci Lab.',
                  tags=['Braun_Lab', 'EZH2_Knockout', 'hematopoietic_stem_cell', 'HiCkory'],
                  species='homo_sapiens',
                  location='/home/cfrankston/Projects/hic_scope/hic_files/KO.hic',
                  genome_assembly_name='GRCh38',
                  hic_type='in_situ'))

informants_list.append(HiC_File(name='EZH2_Mock_Merge.hic',
                  description='Bulk HiC data from the laboratory of Ted Braun at OHSU of hematopoietic stem cells controlling against an EZH2 CRISPR knockout, merged from three technical replicates by the HiCkory authored by PhD. student Benjamin Skubi in the Yardimci Lab.',
                  tags=['Braun_Lab', 'EZH2_Knockout', 'hematopoietic_stem_cell', 'HiCkory'],
                  species='homo_sapiens',
                  location='/home/cfrankston/Projects/hic_scope/hic_files/Mock.hic',
                  genome_assembly_name='GRCh38',
                  hic_type='in_situ'))

#print(KO_hic.__dict__)
#print('\n')

informants_list.append(hicstraw_getMatrixZoomData_Parameters(name='mzd_setting_1',
                                                      parameters={'chr1':1,
                                                                  'chr2':1,
                                                                  'obs_type':'observed',
                                                                  'norm':'VC_SQRT',
                                                                  'resolution_units':'BP',
                                                                  'res':10000}))
#print(my_mzd_params.__dict__)
#(hicstraw_getMatrixZoomData.__dict__)

Mustache = Algorithm()

loop_bedpes_list = informant_class.create_file_informant_list_from_folder(root_folder='/home/cfrankston/Projects/hic_scope/bedpe_files', use_location=True, attribute_sequence=['name'],
                                                                              informant_class=HiC_Loops_File, reference_informant_names=['Mustache'], description='observed_VC_SQRT_5000bp_diff_fdr2_0.05_mustache_fdr1_0.2_results_folder_February_21_2024',
                                                                              genome_assembly_name='GRCh38', gz=False, species='homo_sapiens')

informants_list += (loop_bedpes_list)

this_informant_df.append(informants_list)

this_informant_df.append([HiC_Loops_File(name='KO_VSMock.diffloop1.consensus.bedpe', description='Preliminary consensus loops produced between two Mustache loop calls at different normalizations and fdr rates and an arbitrary consensus score threshold at 10kbp resolution.', reference_informant_names=['Mustache'], tags=['consensus_features', 'EZH2_KO', 'hematopoietic_stem_cells'], genome_assembly_name='GRCh38', gz=False, species='homo_sapiens', location = '/home/cfrankston/Projects/consensus_features/consensus_features/results/KO_VS_Mock.diffloop1.consensus.bedpe')])

print(this_informant_df.df.loc[6]['informant'].__dict__)
this_informant_df.df

{'name': 'KO_VS_Mock.diffloop2', 'description': 'observed_VC_SQRT_5000bp_diff_fdr2_0.05_mustache_fdr1_0.2_results_folder_February_21_2024', 'tags': [], 'reference_informant_names': ['Mustache'], 'informant_class': 'HiC_Loops_File', 'reference_informant_name_redundancy_values': {'Mustache': None}, 'source_depth': 6, 'species': 'homo_sapiens', 'location': '/home/cfrankston/Projects/hic_scope/bedpe_files/KO_VS_Mock.diffloop2', 'external_locations': None, 'file_type': '.bedpe', 'genome_assembly_name': 'GRCh38', 'aliases': None, 'hic_file': None, 'feature_type': 'HiC_Loop', 'gz': False}


,name,informant,entry_time,verification_status
0,EZH2_KO_Merge.hic,<ontologies.hic_January_24_2024_ontology.HiC_F...,06_25_2024,pending
1,EZH2_Mock_Merge.hic,<ontologies.hic_January_24_2024_ontology.HiC_F...,06_25_2024,pending
2,mzd_setting_1,<ontologies.hic_January_24_2024_ontology.hicst...,06_25_2024,pending
3,KO_VS_Mock.loop2,<ontologies.hic_January_24_2024_ontology.HiC_L...,06_25_2024,pending
4,KO_VS_Mock.diffloop1,<ontologies.hic_January_24_2024_ontology.HiC_L...,06_25_2024,pending
5,KO_VS_Mock.loop1,<ontologies.hic_January_24_2024_ontology.HiC_L...,06_25_2024,pending
6,KO_VS_Mock.diffloop2,<ontologies.hic_January_24_2024_ontology.HiC_L...,06_25_2024,pending
7,KO_VSMock.diffloop1.consensus.bedpe,<ontologies.hic_January_24_2024_ontology.HiC_L...,06_25_2024,pending


In [9]:
this_informant_df.df['verification_status'] = True
this_informant_df.filter('isinstance(@informant, HiC_Loops_File)', additional_context={'HiC_Loops_File':HiC_Loops_File})

this_informant_df.save_df(df_pkl_path='/home/cfrankston/Projects/hic_scope/informant_dataframes/hic_scope_test_1.pkl')

Error evaluating expression: isinstance expected 2 arguments, got 1
Error evaluating expression: isinstance expected 2 arguments, got 1
Error evaluating expression: isinstance expected 2 arguments, got 1
Error evaluating expression: isinstance expected 2 arguments, got 1
Error evaluating expression: isinstance expected 2 arguments, got 1
Error evaluating expression: isinstance expected 2 arguments, got 1
Error evaluating expression: isinstance expected 2 arguments, got 1
Error evaluating expression: isinstance expected 2 arguments, got 1


In [12]:
consensus_trial_informant_df = informant_class.Informant_Dataframe()

putative_loops_inf_df = informant_class.create_file_informant_list_from_folder(root_folder = "/home/cfrankston/Projects/CEDAR_Projects/2024-02-21_EZH2-knockout-hic/data/data_March_13_2024",
use_location=True, attribute_sequence=['description','name'],informant_class=HiC_Loops_File, reference_informant_names=['Mustache'],
                                                                              genome_assembly_name='GRCh38', gz=False, species='homo_sapiens')

consensus_trial_informant_df.append(putative_loops_inf_df)
consensus_trial_informant_df.df

consensus_trial_informant_df.save_df(df_pkl_path='/home/cfrankston/Projects/consensus_features/consensus_features/informant_dataframes/loop_rep_infs_df')


In [11]:
consensus_trial_informant_df.df['informant'][0].__dict__

{'name': 'rep1_KO_VS_Mock.diffloop2',
 'description': 'observed_KR_10000bp_diff_fdr2_0.05_mustache_fdr1_0.15_results_folder_March_7_2024',
 'tags': [],
 'reference_informant_names': ['Mustache'],
 'informant_class': 'HiC_Loops_File',
 'reference_informant_name_redundancy_values': {'Mustache': None},
 'source_depth': 6,
 'species': 'homo_sapiens',
 'location': '/home/cfrankston/Projects/CEDAR_Projects/2024-02-21_EZH2-knockout-hic/data/observed_KR_10000bp_diff_fdr2_0.05_mustache_fdr1_0.15_results_folder_March_7_2024/KO_VS_Mock/rep1_KO_VS_Mock.diffloop2',
 'external_locations': None,
 'file_type': '.bedpe',
 'genome_assembly_name': 'GRCh38',
 'aliases': None,
 'hic_file': None,
 'feature_type': 'HiC_Loop',
 'gz': False}

## Launch Project using Imported Ontology and Desired Informant Dataframe

In [14]:
import pandas as pd
import subprocess
import snakemake

project_name = "test_launch_v3/version-1.0"#"PDAC_reprogramming_RNA/version-1.0"
#"project_manager/version_1.0"#"transcription-from-3D-genome_attributions/version_1.0"

informant_class_path = "/home/groups/CEDAR/franksto/ont_rdb/ont_rdb/informant_class.py"

ontology_script_path = '/home/groups/CEDAR/franksto/ont_rdb/ont_rdb/ontologies/hic_January_24_2024_ontology.py'#'/home/groups/CEDAR/franksto/ont_rdb/ont_rdb/ontologies/2024-11-11_RNA_seq_ontology.py'
#"/home/groups/CEDAR/franksto/ont_rdb/ont_rdb/ontologies/2024-8-15_hic_ontology.py"
#"/home/groups/CEDAR/franksto/ont_rdb/ont_rdb/ontologies/project_manager_version_1_ontology.py"

explorer_auxiliaries_path = "/home/groups/CEDAR/franksto/ont_rdb/ont_rdb/explorer_auxiliaries.py"

informant_dataframe_path = "ont_rdb/ont_rdb/ontology_dataframes/hic_January_24_2024_ontology_dataframe.pkl"
base_directory = "/home/groups/CEDAR/franksto"#/2024-6-24"

meta_log = ""

command = f"python  launch_project_3.0.py {project_name} {informant_class_path} {ontology_script_path} {explorer_auxiliaries_path} {informant_dataframe_path} {base_directory}"
result = subprocess.run(command, capture_output=True, text=True, shell=True)

print(f"Processing command: \n{command}")
# Check if the command was successful
if result.returncode == 0:
    # Print the standard output of the command
    print(result.stdout)
    print(f"Project named {project_name} is constructed.")
else:
    # Print the standard error if the command failed
    print("Command failed with error:", result.stderr)

Processing command: 
python  launch_project_3.0.py test_launch_v3/version-1.0 /home/groups/CEDAR/franksto/ont_rdb/ont_rdb/informant_class.py /home/groups/CEDAR/franksto/ont_rdb/ont_rdb/ontologies/hic_January_24_2024_ontology.py /home/groups/CEDAR/franksto/ont_rdb/ont_rdb/explorer_auxiliaries.py ont_rdb/ont_rdb/ontology_dataframes/hic_January_24_2024_ontology_dataframe.pkl /home/groups/CEDAR/franksto
Created project structure.
Linked informant_class script, ontology and informants.
Created metadata file.
Copied consolidate_project.py.
Creating explorer notebook for test_launch_v3/version-1.0 in /home/groups/CEDAR/franksto/test_launch_v3/version-1.0.
Created explorer notebook.

Project named test_launch_v3/version-1.0 is constructed.
